In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from collections import Counter

from sklearn.ensemble import GradientBoostingClassifier

In [4]:
# explicitly require this experimental feature
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
# now you can import normally from ensemble
from sklearn.ensemble import HistGradientBoostingClassifier

In [5]:
DATA='ugrin2020-vehiculo-usado-multiclase/'
TRAIN=DATA+'train.csv'
TEST=DATA+'test.csv'

PREPROCESSED_DATA='preprocessed_data/'
RESULTS='results/'

In [6]:
test = pd.read_csv(TEST)
test_ids = test['id']

In [7]:
# Carga de datos ya procesados
X=np.load(PREPROCESSED_DATA+'binScale.npz')
train = X['arr_0']
label = X['arr_1']
test = X['arr_2']
X.close()

In [8]:
train.shape

(9125, 52)

In [12]:
gb1=GradientBoostingClassifier(n_estimators=450, learning_rate=0.14, subsample=0.9, random_state=6)
gb2=GradientBoostingClassifier(n_estimators=500, learning_rate=0.12, subsample=0.9, random_state=25)
gb3=GradientBoostingClassifier(n_estimators=550, learning_rate=0.1, subsample=0.9, random_state=318)
gb4=GradientBoostingClassifier(n_estimators=600, learning_rate=0.08, subsample=0.9, random_state=4523)
hgb1=HistGradientBoostingClassifier(learning_rate=0.1,max_iter=200)
hgb2=HistGradientBoostingClassifier(learning_rate=0.3,max_iter=100)

lr=LogisticRegression(max_iter=1000)

estimators=[('gb1',gb1),('gb2',gb2),('gb3',gb3),('gb4',gb4),('hgb1',hgb1),('hgb2',hgb2)]
model=StackingClassifier(estimators=estimators,final_estimator=lr,n_jobs=4,verbose=100)

In [6]:
scores=cross_val_score(model, train, label)
print(scores)
print(np.mean(scores))

[0.93972603 0.92657534 0.92931507 0.92657534 0.92986301]
0.9304109589041095


## Generar fichero de Kaggle

In [13]:
model.fit(train,label)
# Ahora predecimos
predict = model.predict(test)
predict = list(map(int,predict))
# Generamos 
df_result = pd.DataFrame({'id': test_ids, 'Precio_cat': predict})
df_result.to_csv(RESULTS+"try20.csv", index=False)